In [ ]:
# Parameters 
# DATASET_FILE_PATH: "BigData.xlsx",
# DATASET_SHEET_TITLE: "data_test",
# GRANULARITY: 10, # Take first item, skip next x, take next, skip x, take next, ...
# STEP_SIZE_SLIDING_WINDOW: 5,
# PAST_HISTORY: 20,
# FUTURE_TARGET: 1, 
# Looks at observations that are PASTHISTORY timesteps prior (reading only every STEP_SIZE_SLIDING_WINDOW entry),
# then predicts the result at FUTURETARGET timesteps in the future
# VAL_PERCENT: 0.1,
# TEST_PERCENT: 0.1,
# EPOCHS: 5,
# BATCH_SIZE: 10 # Splits the dataset into batches of this size: we perform gradiant descent once per batch
# SMOOTHING: 0

# runAll(DATASET_FILE_PATH, DATASET_SHEET_TITLE, GRANULARITY, STEP_SIZE_SLIDING_WINDOW, PAST_HISTORY, 
#        FUTURE_TARGET, VAL_PERCENT, TEST_PERCENT, EPOCHS, BATCH_SPLITS_TRAIN, BATCH_SPLITS_VAL, SMOOTHING)

In [ ]:
# Load algorithm
# TODO?: In the future I might want to split this up further into more files
%run ./lstm_baseline.ipynb

In [ ]:
DATASET_FILE_PATH="Datasets\\BigData.xlsx"
DATASET_SHEET_TITLE="data_test"
GRANULARITY=10
STEP_SIZE_SLIDING_WINDOW=5
PAST_HISTORY=20
FUTURE_TARGET=1
VAL_PERCENT=0.1
TEST_PERCENT=0.1
EPOCHS=5
BATCH_SPLITS=10
SMOOTHING=50
ATTEMPT_NAME="LSTM_BASELINE"
SHUFFLE_BUFFER_SIZE=100

# Running the algorithm all at once
BASELINE = run_all(DATASET_FILE_PATH, DATASET_SHEET_TITLE, GRANULARITY, STEP_SIZE_SLIDING_WINDOW, 
                   PAST_HISTORY, FUTURE_TARGET, VAL_PERCENT, TEST_PERCENT, EPOCHS, BATCH_SPLITS, 
                   SMOOTHING, ATTEMPT_NAME, SHUFFLE_BUFFER_SIZE)

In [ ]:
# Running the algorithm part-by-part
raw_data = load_dataset(DATASET_FILE_PATH, DATASET_SHEET_TITLE, GRANULARITY)
indexes, features, ground_truth = split_data(raw_data, GRANULARITY, SMOOTHING)
plot_dataset(features, ground_truth, ATTEMPT_NAME)
x_train, y_train, x_val, y_val, x_test, y_test = slice_data(indexes, features, ground_truth, VAL_PERCENT, 
                                                            TEST_PERCENT, PAST_HISTORY, FUTURE_TARGET, 
                                                            STEP_SIZE_SLIDING_WINDOW, GRANULARITY)
batched_train_data, batched_val_data, batched_test_data = batch_data(x_train, y_train, x_val, y_val, 
                                                                     x_test, y_test, BATCH_SPLITS, EPOCHS,
                                                                     SHUFFLE_BUFFER_SIZE)
model = compile_model(x_train, FUTURE_TARGET)

In [ ]:
training_history = fit_model(model, x_train, x_val, batched_train_data, batched_val_data, BATCH_SPLITS, EPOCHS)

In [ ]:
plot_results(training_history, model, batched_test_data, indexes, ground_truth)

In [ ]:
evaluate_model(model, batched_test_data)

In [ ]:
# FUTURE WORKS:
# - Modify it such that I can import different sorts of MI algorithms